## api-football.com

- http.client

In [ ]:
import http.client

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "253f4f32af9f0b0741d6705b520322da"
    }

conn.request("GET", "/leagues", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

In [ ]:
https://media.api-sports.io/football/teams/{team_id}.png

In [ ]:
import requests

url = "https://media.api-sports.io/football/teams/19.png"

payload={}
headers = {
  'x-rapidapi-key': '253f4f32af9f0b0741d6705b520322da',
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

1. How Your Existing System Works

- id: A unique auto‑incrementing API identifier.
- api_name: The name of the API (e.g., “football-data.org”).
- category: A broad classification (e.g., “sports” or “entertainment”).
- subcategory: A more refined classification (e.g., “football” or “gaming”).
- base_url: The base endpoint of the API.
- authentication: Indicates the type of authentication required (for example: API key, OAuth, etc.).
- request_example: An example of how to send a request.
- response_format: Format of the response (e.g., “json”).
- rate_limit: Limits on how often the API can be called.
- docs_url: A link to documentation.
- api_status: Whether the API is active.
- access_token: The token used for authentication.
- token_expiry: When the token expires.
- environment_var: (Perhaps used in the past for environment-based configuration)
- last_updated: Timestamp for the last update.

2. Updated Python Code (Using SQLAlchemy and Pandas)

Below is the updated Python script. (Make sure you have the required packages installed: pandas, requests, and sqlalchemy along with a MySQL‑compatible DB driver like pymysql.) Update the connection string in the code with your actual database credentials

In [1]:
import pandas as pd
import requests
import json
from sqlalchemy import create_engine

# Connection string to your MariaDB “apidb” database.
# Replace YOUR_PASSWORD with your actual MySQL/MariaDB password.
DB_CONNECTION_STRING = 'mysql+pymysql://root:@localhost/apidb'

def load_api_dataframe():
    """
    Load API data from the database's api_details table into a Pandas DataFrame.
    """
    try:
        engine = create_engine(DB_CONNECTION_STRING)
        query = "SELECT * FROM api_details"
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print("Error loading API details from database:", e)
        return pd.DataFrame()

def list_categories(df):
    """
    Return unique API categories (from the 'category' field).
    """
    return df["category"].unique().tolist()

def list_subcategories(df, category):
    """
    Return unique subcategories for a given category (filtered on 'subcategory').
    """
    filtered = df[df["category"].str.lower() == category.lower()]
    return filtered["subcategory"].unique().tolist()

def fetch_api_data(df, category, subcategory, api_name, method="GET", payload=None):
    """
    Given a category, subcategory, and api_name, fetch data from the target API.
    
    Fields used:
      - base_url: Used as the API endpoint.
      - access_token: If provided, for Bearer token authentication.
    
    The function builds an HTTP request using the specified method.
    """
    # Filter records making the matches case-insensitive.
    api_entry = df[
        (df["category"].str.lower() == category.lower()) &
        (df["subcategory"].str.lower() == subcategory.lower()) &
        (df["api_name"].str.lower() == api_name.lower())
    ]
    
    if api_entry.empty:
        return {"error": f"API '{api_name}' not found in '{category} > {subcategory}'!"}

    try:
        row = api_entry.iloc[0]
        url = row["base_url"]  # Use the base_url field as the endpoint

        # Build headers; add the Authorization header only if access_token exists.
        headers = {"Content-Type": "application/json"}
        if pd.notnull(row["access_token"]) and row["access_token"].strip() != "":
            headers["Authorization"] = f"Bearer {row['access_token']}"
        
        method = method.upper()
        if method == "GET":
            response = requests.get(url, headers=headers, timeout=5)
        elif method == "POST":
            response = requests.post(url, json=payload, headers=headers, timeout=5)
        elif method == "PUT":
            response = requests.put(url, json=payload, headers=headers, timeout=5)
        elif method == "DELETE":
            response = requests.delete(url, headers=headers, timeout=5)
        else:
            return {"error": f"Invalid HTTP method: {method}"}

        response.raise_for_status()
        return response.json()  # Return the parsed JSON data
    except requests.exceptions.RequestException as e:
        return {"error": f"Failed to fetch data: {str(e)}"}

def api_manager():
    """
    CLI-based API management that loads API details from the database.
    Users can choose a category, subcategory, and API name, then attempt a GET request.
    """
    df = load_api_dataframe()
    if df.empty:
        print("No API details loaded from the database.")
        return
    
    while True:
        print("\n=== API Manager ===")
        categories = list_categories(df)
        print("Available Categories:", ", ".join(categories))
        category = input("Select a category or type 'exit': ").strip().lower()
        if category == "exit":
            print("Closing API Manager...")
            break
        if category not in [c.lower() for c in categories]:
            print(f"Error: Category '{category}' not found!")
            continue

        subcategories = list_subcategories(df, category)
        print("Available Subcategories:", ", ".join(subcategories))
        subcategory = input("Select a subcategory: ").strip().lower()
        
        # Filter based on category and subcategory to list available API names.
        filtered = df[
            (df["category"].str.lower() == category.lower()) &
            (df["subcategory"].str.lower() == subcategory.lower())
        ]
        available_apis = filtered["api_name"].tolist()
        print("Available APIs:", ", ".join(available_apis))
        api_name = input("Select API: ").strip().lower()

        data = fetch_api_data(df, category, subcategory, api_name, method="GET")
        print(json.dumps(data, indent=4))

if __name__ == "__main__":
    api_manager()


Error loading API details from database: No module named 'pymysql'
No API details loaded from the database.


In [22]:
!pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 40.2 MB/s eta 0:00:00


In [5]:
!conda list

# packages in environment at C:\Users\Richard\.conda\envs\football_env:
#
# Name                    Version                   Build  Channel
anyio                     4.9.0                    pypi_0    pypi
argon2-cffi               23.1.0                   pypi_0    pypi
argon2-cffi-bindings      21.2.0                   pypi_0    pypi
arrow                     1.3.0                    pypi_0    pypi
asttokens                 3.0.0                    pypi_0    pypi
async-lru                 2.0.5                    pypi_0    pypi
attrs                     25.3.0                   pypi_0    pypi
babel                     2.17.0                   pypi_0    pypi
beautifulsoup4            4.13.4                   pypi_0    pypi
bleach                    6.2.0                    pypi_0    pypi
ca-certificates           2025.2.25            haa95532_0  
certifi                   2025.1.31                pypi_0    pypi
cffi                      1.17.1                   pypi_0    pypi
charset

3. How This Updated Script Uses Your Table Structure

Loading Data: The function load_api_dataframe() connects to your MySQL/MariaDB database apidb (using SQLAlchemy) and selects all rows from the api_details table. This DataFrame will have all columns using the names from your table (for example, category, api_name, base_url, and access_token).

Listing Categories and Subcategories: The functions list_categories() and list_subcategories() use the category and subcategory fields to let users pick which API they want to work with.

Fetching API Data: The function fetch_api_data() looks up the API by matching the lower-case form of the fields (category, subcategory, and api_name). It then uses the base_url as the endpoint and, if available, adds an HTTP header using the access_token (forming a Bearer token). The function then issues a GET (or other method) request and returns parsed JSON data or an error.

CLI Interaction: In api_manager(), the script continuously prompts the user to select a category, subcategory, and API—then attempts to call the API and prints its result.